# 1. Imports

In [ ]:
# System, file operations, and utilities
import os
import re
import glob
import shutil
import random
import yaml
import time
from pathlib import Path
from datetime import datetime
from threading import Thread

# Data manipulation and numerical operations
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# Image processing and augmentation
import cv2
import albumentations as A

# Machine Learning and Computer Vision
from ultralytics import YOLO

# Visualization and Interactive UI (Jupyter/Colab)
import matplotlib.pyplot as plt
from IPython.display import display, Image, clear_output
import ipywidgets as widgets
import matplotlib.dates as mdates




# 2. Merging datasets

## Seaship

In [ ]:
source = 'sea_ship/seaship.v1i.yolov8/test/images'
source_1 = 'sea_ship/seaship.v1i.yolov8/train/images'
destination = 'merged_dataset/images'

# gather all files
allfiles = os.listdir(source)
allfiles_1 = os.listdir(source_1)

# iterate on all files to move them to destination folder
for f in allfiles:
    src_path = os.path.join(source, f)
    dst_path = os.path.join(destination, f)
    os.rename(src_path, dst_path)

for f in allfiles_1:
    src_path = os.path.join(source_1, f)
    dst_path = os.path.join(destination, f)
    os.rename(src_path, dst_path)


In [ ]:
source_labels = 'sea_ship/seaship.v1i.yolov8/test/labels'
source_1_labels = 'sea_ship/seaship.v1i.yolov8/train/labels'
destination = 'merged_dataset/labels'

# gather all files
allfiles = os.listdir(source_labels)
allfiles_1 = os.listdir(source_1_labels)

# iterate on all files to move them to destination folder
for f in allfiles:
    src_path = os.path.join(source_labels, f)
    dst_path = os.path.join(destination, f)
    os.rename(src_path, dst_path)

for f in allfiles_1:
    src_path = os.path.join(source_1_labels, f)
    dst_path = os.path.join(destination, f)
    os.rename(src_path, dst_path)

In [ ]:
class_mapping = {i: 0 for i in range(9)}

input_dir = r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\seaships\valid\labels"
output_dir = r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\seaships\valid\labels_new"

os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.txt'):
        with open(os.path.join(input_dir, filename), 'r') as f_in, \
             open(os.path.join(output_dir, filename), 'w') as f_out:
            for line in f_in:
                parts = line.strip().split()
                if parts:
                    old_class = int(parts[0])
                    new_class = class_mapping.get(old_class, old_class)
                    parts[0] = str(new_class)
                    f_out.write(' '.join(parts) + '\n')

## Singapore Maritime Dataset

### Replacing numbering of the labels

In [ ]:
class_mapping = {i: 0 for i in range(20)}

input_dir = "Singapore maritime.v5i.yolov8/test/labels"
output_dir = "Singapore maritime.v5i.yolov8/test/labels_new"

os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.txt'):
        with open(os.path.join(input_dir, filename), 'r') as f_in, \
             open(os.path.join(output_dir, filename), 'w') as f_out:
            for line in f_in:
                parts = line.strip().split()
                if parts:
                    old_class = int(parts[0])
                    new_class = class_mapping.get(old_class, old_class)
                    parts[0] = str(new_class)
                    f_out.write(' '.join(parts) + '\n')

## Moving the images and labels to the desired location

In [ ]:
source = "Singapore maritime.v5i.yolov8/train/images"
source_1 = "Singapore maritime.v5i.yolov8/valid/images"
source_2 = "Singapore maritime.v5i.yolov8/test/images"
destination = "merged_dataset/images"

# gather all files
allfiles = os.listdir(source)
allfiles_1 = os.listdir(source_1)
allfiles_2 = os.listdir(source_2)

# iterate on all files to move them to destination folder
for f in allfiles:
    src_path = os.path.join(source, f)
    dst_path = os.path.join(destination, f)
    os.rename(src_path, dst_path)

for f in allfiles_1:
    src_path = os.path.join(source_1, f)
    dst_path = os.path.join(destination, f)
    os.rename(src_path, dst_path)

for f in allfiles_2:
    src_path = os.path.join(source_2, f)
    dst_path = os.path.join(destination, f)
    os.rename(src_path, dst_path)


In [ ]:
source = "Singapore maritime.v5i.yolov8/train/labels_new"
source_1 = "Singapore maritime.v5i.yolov8/valid/labels_new"
source_2 = "Singapore maritime.v5i.yolov8/test/labels_new"
destination = "merged_dataset/labels"

# gather all files
allfiles = os.listdir(source)
allfiles_1 = os.listdir(source_1)
allfiles_2 = os.listdir(source_2)

# iterate on all files to move them to destination folder
for f in allfiles:
    src_path = os.path.join(source, f)
    dst_path = os.path.join(destination, f)
    os.rename(src_path, dst_path)

for f in allfiles_1:
    src_path = os.path.join(source_1, f)
    dst_path = os.path.join(destination, f)
    os.rename(src_path, dst_path)

for f in allfiles_2:
    src_path = os.path.join(source_2, f)
    dst_path = os.path.join(destination, f)
    os.rename(src_path, dst_path)

# 2. Data exploration

## Building up the dataframe w/ current files

In [ ]:

# Directory containing label .txt files
labels_dir = 'merged_dataset/labels'
files = sorted(glob.glob(os.path.join(labels_dir, '*.txt')))

rows = []
if not files:
    # No label files found: create an empty dataframe with expected columns
    print('No label files found in', labels_dir)
    labels_df = pd.DataFrame(columns=['label_file', 'image_file', 'class', 'x', 'y', 'w', 'h'])
else:
    for fp in files:
        basename = os.path.basename(fp)
        image_name = os.path.splitext(basename)[0] + '.jpg'
        # read non-empty lines
        with open(fp, 'r', encoding='utf-8') as f:
            lines = [ln.strip() for ln in f.readlines() if ln.strip() != '']
        if not lines:
            # file had no labels; create one row with NaNs for class and coords
            rows.append({
                'label_file': basename,
                'image_file': image_name,
                'class': pd.NA,
                'x': pd.NA,
                'y': pd.NA,
                'w': pd.NA,
                'h': pd.NA
            })
        else:
            for ln in lines:
                parts = ln.split()
                # YOLO format: class x_center y_center width height
                try:
                    cls = int(parts[0]) if len(parts) >= 1 else pd.NA
                except ValueError:
                    cls = pd.NA
                coords = [pd.NA, pd.NA, pd.NA, pd.NA]
                if len(parts) >= 5:
                    try:
                        coords = [float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])]
                    except ValueError:
                        coords = [pd.NA, pd.NA, pd.NA, pd.NA]
                rows.append({
                    'label_file': basename,
                    'image_file': image_name,
                    'class': cls,
                    'x': coords[0],
                    'y': coords[1],
                    'w': coords[2],
                    'h': coords[3],
                })
    # build dataframe from parsed rows
    labels_df = pd.DataFrame(rows)

# compute summary stats
total_files = len(files)
total_boxes = int(labels_df['class'].notna().sum()) if not labels_df.empty else 0
empty_label_files = int(labels_df.loc[labels_df['class'].isna(), 'label_file'].nunique()) if not labels_df.empty else 0
print(f'Total label files scanned: {total_files}')
print(f'Total bounding boxes (rows): {total_boxes}')
print(f'Empty label files: {empty_label_files}')

# show a sample preview
labels_df.head(10)

## Summing up and plotting the class distribution

In [ ]:
# Try to load class names from merged_dataset/data.yaml
class_names = {}
data_yaml_path = 'merged_dataset/data.yaml'
if os.path.exists(data_yaml_path):
    try:
        import yaml
        with open(data_yaml_path, 'r', encoding='utf-8') as f:
            data = yaml.safe_load(f)
        names = data.get('names') if isinstance(data, dict) else None
        if isinstance(names, dict):
            # keys may be strings; convert to ints
            class_names = {int(k): v for k, v in names.items()}
        elif isinstance(names, list):
            class_names = {i: n for i, n in enumerate(names)}
    except Exception:
        # fallback: simple parse for a YAML 'names' block
        try:
            with open(data_yaml_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
            names_list = []
            in_names = False
            for ln in lines:
                s = ln.strip()
                if s.startswith('names:'):
                    # might be inline list or start of block
                    rest = s.split('names:', 1)[1].strip()
                    if rest.startswith('['):
                        # literal list, try eval safely
                        import ast
                        try:
                            names_list = ast.literal_eval(rest)
                        except Exception:
                            names_list = []
                        break
                    else:
                        in_names = True
                        continue
                if in_names:
                    if s.startswith('-'):
                        names_list.append(s.lstrip('-').strip().strip('"'))
                    else:
                        break
            class_names = {i: n for i, n in enumerate(names_list)}
        except Exception:
            class_names = {}
else:
    print(f'No data.yaml at {data_yaml_path}; falling back to numeric class IDs')

# If labels_df is empty or has no class entries, show message
if 'labels_df' not in globals() or labels_df.empty or labels_df['class'].dropna().empty:
    print('No labeled bounding boxes to plot.')
else:
    counts = labels_df['class'].dropna().astype(int).value_counts().sort_index()
    idx = list(counts.index)
    vals = counts.values
    x_labels = [class_names.get(i, str(i)) for i in idx]
    plt.figure(figsize=(12,6))
    bars = plt.bar(x_labels, vals, color='tab:blue')
    plt.title('Class Distribution')
    plt.ylabel('Number of bounding boxes')
    plt.xlabel('Class')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    # annotate counts on bars
    for bar in bars:
        h = bar.get_height()
        plt.annotate(f'{int(h)}', xy=(bar.get_x() + bar.get_width() / 2, h), xytext=(0, 3), textcoords='offset points', ha='center', va='bottom', fontsize=9)
    plt.tight_layout()
    plt.show()


## Creating video with ground truth

In [ ]:
# Paths - adjust these to your setup
images_dir = "merged_dataset/images"
labels_dir = "merged_dataset/labels"
output_video = "ground_truth_video_singapore_maritime.mp4"

# Class names dictionary (adjust based on your dataset)
class_names = {
    0: "bulk cargo carrier",
    1: "container ship",
    2: "fishing boat",
    3: "general cargo ship",
    4: "ore carrier",
    5: "passenger ship",
    6: "Boat",
    7: "Buoy",
    8: "Ferry",
    9: "Flying bird-plane",
    10: "Kayak",
    11: "Other",
    12: "Sail boat",
    13: "Speed boat",
    14: "Vessel-ship",
}

# Filter only Singapore Maritime dataset images (they start with 'MVI')
image_files = sorted(glob.glob(os.path.join(images_dir, "MVI*.jpg")))
# Try other extensions if needed
if not image_files:
    image_files = sorted(glob.glob(os.path.join(images_dir, "MVI*.png")))

# Check if we found any images
if not image_files:
    print("No Singapore Maritime dataset images found!")
else:
    # Read first image to get dimensions
    first_img = cv2.imread(image_files[0])
    height, width = first_img.shape[:2]

    # Define the output video writer using mp4v codec, 30fps
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, 30.0, (width, height))

    # Process each image
    print(f"Processing {len(image_files)} images...")

    for idx, img_path in enumerate(image_files):
        # Print progress message
        if (idx + 1) % 10 == 0:
            print(f"Processed {idx + 1}/{len(image_files)} images")

        # Read the image
        img = cv2.imread(img_path)

        # Skip if image couldn't be loaded
        if img is None:
            print(f"Warning: Could not load {img_path}")
            continue

        # Get corresponding label file
        img_name = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(labels_dir, f"{img_name}.txt")

        # Get all bounding boxes for this image
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()

                    # Skip if no class information
                    if len(parts) < 5:
                        continue

                    class_id = int(parts[0])
                    x_center, y_center, box_width, box_height = map(
                        float, parts[1:5]
                    )

                    # Convert YOLO format to pixel coordinates
                    x_center_px = int(x_center * width)
                    y_center_px = int(y_center * height)
                    box_width_px = int(box_width * width)
                    box_height_px = int(box_height * height)

                    # Calculate the corner points from center, width, height
                    x1 = int(x_center_px - box_width_px / 2)
                    y1 = int(y_center_px - box_height_px / 2)
                    x2 = int(x_center_px + box_width_px / 2)
                    y2 = int(y_center_px + box_height_px / 2)

                    # Draw rectangle (green color)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    # Get class name if available from the class_names dict
                    class_label = class_names.get(class_id, f"Class_{class_id}")

                    # Put class name text above the box
                    cv2.putText(
                        img, class_label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2
                    )

        # Write frame to video
        out.write(img)

    # Release the video writer
    out.release()
    #print(f"Video saved to {output_video}")
    #print(f"Total frames: {len(image_files)}")

In [ ]:
for label, label_name in class_names.items():
    print(f"Class ID: {label}, Class Name: {label_name}")

In [ ]:
labels_df = labels_df.loc[(labels_df['class'] != 7) & (labels_df['class'] != 9) & (labels_df['class'] != 11) & (labels_df['class'].notna())]
labels_df['class'].replace({4:0, 5:0, 1:0, 0:0, 3:0, 2:0, 14:0, 6:1, 13:1, 8:1, 12:1, 10:1}, inplace=True)

In [ ]:
labels_df = labels_df.loc[(labels_df['class'] != 7) & (labels_df['class'] != 9) & (labels_df['class'] != 11) & (labels_df['class'].notna())]
labels_df['class'].replace({1:0}, inplace=True)

# Image preparation for the training

## Dividing data (SMD + SeaShip) into train and test datasets

In [ ]:
train_set = labels_df.sample(frac=0.7, random_state=42)

# Dropping all those indexes from the dataframe that exists in the train_set
test_set = labels_df.drop(train_set.index)
train_set.shape, test_set.shape

valid_set = test_set.sample(frac=0.33, random_state=42)
test_set = test_set.drop(valid_set.index)
train_set.shape, test_set.shape, valid_set.shape

In [ ]:
valid_set

## Creating folders for train, test, valid splits

In [ ]:
for file in train_set['image_file'].unique():
    src_path = os.path.join('merged_dataset/images', file)
    dst_path = os.path.join('final_dataset/train/images', file)
    os.makedirs(os.path.dirname(dst_path), exist_ok=True)
    shutil.copy2(src_path, dst_path)

for file_label in train_set['label_file'].unique():
    src_path = os.path.join('merged_dataset/labels', file_label)
    dst_path = os.path.join('final_dataset/train/labels', file_label)
    os.makedirs(os.path.dirname(dst_path), exist_ok=True)
    shutil.copy2(src_path, dst_path)

for file in valid_set['image_file'].unique():
    src_path = os.path.join('merged_dataset/images', file)
    dst_path = os.path.join('final_dataset/valid/images', file)
    os.makedirs(os.path.dirname(dst_path), exist_ok=True)
    shutil.copy2(src_path, dst_path)

for file_label in valid_set['label_file'].unique():
    src_path = os.path.join('merged_dataset/labels', file_label)
    dst_path = os.path.join('final_dataset/valid/labels', file_label)
    os.makedirs(os.path.dirname(dst_path), exist_ok=True)
    shutil.copy2(src_path, dst_path)

for file in test_set['image_file'].unique():
    src_path = os.path.join('merged_dataset/images', file)
    dst_path = os.path.join('final_dataset/test/images', file)
    os.makedirs(os.path.dirname(dst_path), exist_ok=True)
    shutil.copy2(src_path, dst_path)

for file_label in test_set['label_file'].unique():
    src_path = os.path.join('merged_dataset/labels', file_label)
    dst_path = os.path.join('final_dataset/test/labels', file_label)
    os.makedirs(os.path.dirname(dst_path), exist_ok=True)
    shutil.copy2(src_path, dst_path)

In [ ]:
import gc
gc.collect()
t = gc.get_threshold()
t

## Training the yolo model w/out augmentation (Seaships + SMD)

In [ ]:
# Load a COCO-pretrained YOLOv8s model
model = YOLO("yolov8s.pt")

# Display model information (optional)
model.info()

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data="final_dataset/data.yaml", epochs=10, imgsz=640)
results

###
### DID NOT WORK WELL ENOUGH

In [ ]:
import gc
gc.collect()

## Fine tuning

In [ ]:
# Model Loading
model = YOLO(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\runs\detect\train\weights\best.pt")

# Training with aggressive augmentation for difficult lighting conditions
results = model.train(
    data="fine_set_last/data.yaml",
    epochs=15,          # Short fine-tuning phase
    imgsz=1280,         # High resolution (effective for small details on the bridge)

    # --- PHOTOMETRIC AUGMENTATIONS (Key for sunlight handling) ---
    hsv_h=0.015,        # Hue adjustment (subtle)
    hsv_s=0.7,          # Saturation adjustment (aggressive - sunlight alters colors)
    hsv_v=0.6,          # Brightness adjustment (aggressive - simulating shadows and overexposure)

    # --- GEOMETRIC AUGMENTATIONS ---
    degrees=5.0,        # Slight rotation (simulates camera sway in the wind)
    translate=0.1,      # Image translation/shift
    scale=0.5,          # Scaling (important for varying object distances)
    fliplr=0.5,         # Horizontal flip (almost always beneficial)

    # --- SPECIAL AUGMENTATIONS ---
    mosaic=1.0,         # Enabled (improves context learning)
    mixup=0.1,          # Image blending (optional, helps with dense crowds/details)
    batch=-1,           # Auto-batch size (uses maximum available GPU memory)
)

## All options with the fine tunings below

### Main model development

In [ ]:
# Load a COCO-pretrained YOLOv8s model
model = YOLO("yolov8n.pt")

# Display model information (optional)
model.info()

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data=r"final_dataset\data.yaml", epochs=10, imgsz=640)
results

metrics = model.val(split='test')
metrics

### Mixed-dataset based fine-tuning

In [ ]:
# Model Loading
model = YOLO(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\runs\detect\train4\weights\best.pt")

# Training with aggressive augmentation for difficult lighting conditions
results = model.train(
    data="fine_set_last/data.yaml",
    epochs=15,          # Short fine-tuning phase
    imgsz=1280,         # High resolution (effective for small details on the bridge)
    batch=-1,           # Auto-batch size (uses maximum available GPU memory)

    # --- PHOTOMETRIC AUGMENTATIONS (Critical for harsh sunlight) ---
    hsv_h=0.015,        # Hue adjustment (subtle)
    hsv_s=0.7,          # Saturation adjustment (aggressive - sunlight alters color perception)
    hsv_v=0.6,          # Brightness adjustment (aggressive - simulates shadows and overexposure)

    # --- GEOMETRIC AUGMENTATIONS ---
    degrees=5.0,        # Slight rotation (simulates camera sway in the wind)
    translate=0.1,      # Image translation/shifting
    scale=0.5,          # Scaling (important when objects vary in distance)
    fliplr=0.5,         # Horizontal flip (almost always improves robustness)

    # --- SPECIAL AUGMENTATIONS ---
    mosaic=1.0,         # Enabled (helps model learn objects in different contexts)
    mixup=0.1,          # Image blending (optional, helps with occlusion/dense details)
)

In [ ]:
metrics = model.val(split='test')
metrics

#### Camera_1 validation

In [ ]:
model = YOLO(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\runs\detect\train4\weights\best.pt")

results = model.val(data=r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\camera_1\data.yaml", imgsz=1240)
results

#### Camera_2 validation

In [ ]:
model = YOLO(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\runs\detect\train4\weights\best.pt")

results = model.val(data=r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\camera_2\data.yaml", imgsz=1240)
results

### SeaShip pre-training

In [ ]:
# Load a COCO-pretrained YOLOv8s model
model = YOLO("yolov8n.pt")

# Display model information (optional)
model.info()

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data=r"seaships\data.yaml", epochs=10, imgsz=640)
results

metrics = model.val(split='test')
metrics

### Sea-ship model fine-tuning

In [ ]:
# Model Loading
model = YOLO(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\runs\detect\train3\weights\best.pt")

# Training with aggressive augmentation for difficult lighting conditions
results = model.train(
    data="fine_set_last/data.yaml",
    epochs=15,          # Short fine-tuning phase
    imgsz=1280,         # High resolution (effective for small details on the bridge)

    # --- PHOTOMETRIC AUGMENTATIONS (Critical for handling sunlight) ---
    hsv_h=0.015,        # Hue adjustment (subtle)
    hsv_s=0.7,          # Saturation adjustment (aggressive - sunlight alters color perception)
    hsv_v=0.6,          # Brightness adjustment (aggressive - simulates shadows and overexposure)

    # --- GEOMETRIC AUGMENTATIONS ---
    degrees=5.0,        # Slight rotation (simulates camera sway in the wind)
    translate=0.1,      # Image translation/shifting
    scale=0.5,          # Scaling (important when objects vary in distance/depth)
    fliplr=0.5,         # Horizontal flip (almost always improves robustness)

    # --- SPECIAL AUGMENTATIONS ---
    mosaic=1.0,         # Enabled (improves context learning)
    mixup=0.1,          # Image blending/overlay (helps with dense crowds or overlapping details)
    batch=-1,           # Auto-batch size (optimizes based on available GPU memory)
)

In [ ]:
metrics = model.val(split='test')
metrics

### Singapore Maritime Dataset pre-training

In [ ]:
# Load a COCO-pretrained YOLOv8s model
model = YOLO("yolov8n.pt")

# Display model information (optional)
model.info()

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data=r"Singapore maritime.v5i.yolov8\data.yaml", epochs=10, imgsz=640)
results

metrics = model.val(split='test')
metrics

### Fine-tuning

In [ ]:
# Load the model
model = YOLO(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\runs\detect\train\weights\best.pt")

# Training with aggressive augmentation for difficult lighting conditions
results = model.train(
    data="fine_set_last/data.yaml",
    epochs=15,          # Short fine-tuning
    imgsz=1280,         # High resolution (beneficial for small details on the bridge)

    # --- Photometric Augmentations (Critical for sunlight/shadows) ---
    hsv_h=0.015,        # Hue adjustment (slight)
    hsv_s=0.7,          # Saturation adjustment (aggressive - sun changes colors)
    hsv_v=0.6,          # Value/Brightness adjustment (aggressive - simulates shadows and overexposure)

    # --- Geometric Augmentations ---
    degrees=5.0,        # Slight rotation (simulating camera sway due to wind)
    translate=0.1,      # Image translation
    scale=0.5,          # Scaling (important for varying object distances)
    fliplr=0.5,         # Horizontal flip (almost always useful)

    # --- Special Augmentations ---
    mosaic=1.0,         # Mosaic augmentation (helps learn context)
    mixup=0.1,          # MixUp (image blending, optional)
    batch=-1,           # Auto-batch size
)

In [ ]:
metrics = model.val(split='test')
metrics

In [ ]:
import gc
gc.collect()

In [ ]:
# ================= CONFIGURATION =================
# Update these paths to match your actual local setup
SOURCE_ROOT = Path(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo")
OUTPUT_ROOT = Path(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\Processed_Dataset")

CAMERAS = ["camera_1", "camera_2"]
SUBSETS = ["train", "test", "valid"]

# Regex to find date/time in filename: frame_20251213_135019_...
FILENAME_PATTERN = re.compile(r"frame_(\d{8})_(\d{6})_")

def extract_metadata(filename):
    """Parses filename for datetime and converts to timestamp."""
    match = FILENAME_PATTERN.search(filename)
    if match:
        date_str, time_str = match.groups()
        dt_obj = datetime.strptime(f"{date_str}{time_str}", "%Y%m%d%H%M%S")
        return dt_obj, dt_obj.timestamp()
    return None, None

def count_objects(label_path):
    """Counts lines in a YOLO txt file (1 line = 1 object)."""
    if not os.path.exists(label_path):
        return 0
    with open(label_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
        return len(lines)

def get_unique_filename(directory, filename, extension):
    """
    Checks if a file exists. If so, adds _1, _2, etc.
    Returns the full path including the unique filename.
    """
    base_name = filename
    counter = 0
    while True:
        if counter == 0:
            candidate = f"{base_name}{extension}"
        else:
            candidate = f"{base_name}_{counter}{extension}"

        full_path = directory / candidate
        if not full_path.exists():
            return full_path
        counter += 1

def main():
    data_records = []

    print(f"--- Starting Processing from {SOURCE_ROOT} ---")

    for camera in CAMERAS:
        for subset in SUBSETS:
            img_dir = SOURCE_ROOT / camera / subset / "images"
            lbl_dir = SOURCE_ROOT / camera / subset / "labels"

            if not img_dir.exists():
                continue

            image_files = list(img_dir.glob("*.jpg"))

            for img_path in image_files:
                # 1. Extract Info
                dt_obj, timestamp = extract_metadata(img_path.name)

                if not dt_obj:
                    print(f"Skipping {img_path.name}: Date parsing failed")
                    continue

                date_folder_str = dt_obj.strftime("%Y-%m-%d")

                # 2. Find corresponding label
                label_name = img_path.stem + ".txt"
                label_path = lbl_dir / label_name

                ship_count = count_objects(label_path)

                # 3. Store Data
                data_records.append({
                    "Camera": camera,
                    "Subset": subset,
                    "Original_Filename": img_path.name,
                    "Unix_Timestamp": int(timestamp), # Convert float to int
                    "Date_Str": date_folder_str,
                    "Hour": dt_obj.hour,
                    "Ship_Count": ship_count,
                    "Src_Img": img_path,
                    "Src_Lbl": label_path
                })

    df = pd.DataFrame(data_records)

    if df.empty:
        print("No images found to process.")
        return

    # ================= TASK 1: RESTRUCTURE & RENAME =================
    print("\n--- Restructuring and Renaming Files ---")

    files_moved = 0
    for _, row in df.iterrows():
        # Define target folders: output/camera/date/images/
        target_base = OUTPUT_ROOT / row['Camera'] / row['Date_Str']
        target_img_dir = target_base / "images"
        target_lbl_dir = target_base / "labels"

        # Create dirs
        target_img_dir.mkdir(parents=True, exist_ok=True)
        target_lbl_dir.mkdir(parents=True, exist_ok=True)

        # Generate UNIX Filename
        unix_name = str(row['Unix_Timestamp'])

        # Check for collisions and get unique path
        dest_img_path = get_unique_filename(target_img_dir, unix_name, ".jpg")

        # Determine the final name used (might have _1 appended)
        final_stem = dest_img_path.stem
        dest_lbl_path = target_lbl_dir / f"{final_stem}.txt"

        # Copy Image
        shutil.copy2(row['Src_Img'], dest_img_path)

        # Copy Label (if exists)
        if os.path.exists(row['Src_Lbl']):
            shutil.copy2(row['Src_Lbl'], dest_lbl_path)

        files_moved += 1

    print(f"Successfully processed and renamed {files_moved} files to {OUTPUT_ROOT}")

    # ================= TASK 2: SUMMARY ((images, annotations, days, hours)) =================
    print("\n--- Detailed Summary ---")
    detailed_summary = df.groupby('Camera').agg(
        Images=('Original_Filename', 'count'),
        Annotations=('Ship_Count', 'sum'),
        Unique_Days=('Date_Str', 'nunique'),
        Unique_Hours=('Hour', 'nunique')
    ).reset_index()

    detailed_summary.columns = ['Camera', 'Images', 'Annotations', 'Days', 'Hours']
    print(detailed_summary.to_string(index=False))

    # ================= TASK 3: DATASET REQ (Assorted Date) =================
    print("\n--- Final Dataset Request ---")

    final_report = df.groupby('Camera').agg(
        Images=('Original_Filename', 'count'),
        Ships=('Ship_Count', 'sum'),
        Unique_Dates=('Date_Str', 'unique')
    ).reset_index()

    def format_date_column(dates):
        if len(dates) > 1:
            return "assorted"
        elif len(dates) == 1:
            return dates[0]
        else:
            return "N/A"

    final_report['Date'] = final_report['Unique_Dates'].apply(format_date_column)
    final_output = final_report[['Camera', 'Images', 'Ships', 'Date']]

    print(final_output.to_string(index=False))

    # Save CSV
    final_output.to_csv(OUTPUT_ROOT / "dataset_summary.csv", index=False)

if __name__ == "__main__":
    main()

In [ ]:
SOURCE_ROOT = Path(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\Processed_Dataset")
CAMERAS = ["camera_1", "camera_2"]

def get_timestamps(camera_name):
    print(f"Fetching data for {camera_name}...")
    path = SOURCE_ROOT / camera_name
    timestamps = []
    
    # Extract filename (without .jpg) and convert to int
    # Handles names like: 123456.jpg and 123456_1.jpg
    for f in path.rglob("*.jpg"):
        stem = f.stem.split('_')[0] # Removes potential _1 suffix
        if stem.isdigit():
            timestamps.append(int(stem))
    return sorted(list(set(timestamps))) # Sort for easier comparison

def main():
    # 1. Get sorted timestamp lists
    t1 = get_timestamps(CAMERAS[0])
    t2 = get_timestamps(CAMERAS[1])

    if not t1 or not t2:
        print("No data found in folders!")
        return

    print(f"\nFound: Cam1={len(t1)}, Cam2={len(t2)}")

    # 2. Find nearest neighbors
    # Logic: For every timestamp T in Cam1, find the nearest T in Cam2

    deltas = []
    matched_counts = {
        "exact": 0,      # 0s difference
        "close_1s": 0,   # <= 1s difference
        "close_3s": 0,   # <= 3s difference
        "far": 0         # > 3s difference
    }

    # Convert to array for performance
    arr2 = np.array(t2)

    print("Analyzing time differences...")
    for val1 in t1:
        # Find index where val1 should be inserted in sorted arr2 to maintain order
        idx = np.searchsorted(arr2, val1, side="left")

        # Check neighbors (left and right) to find the absolute nearest value
        candidates = []
        if idx < len(arr2): candidates.append(arr2[idx])
        if idx > 0: candidates.append(arr2[idx-1])

        if not candidates:
            continue

        # Choose the best candidate (smallest absolute difference)
        closest_val2 = min(candidates, key=lambda x: abs(x - val1))
        diff = closest_val2 - val1 # If negative, Cam2 timestamp is earlier
        abs_diff = abs(diff)

        deltas.append(diff)

        if abs_diff == 0: matched_counts["exact"] += 1
        elif abs_diff <= 1: matched_counts["close_1s"] += 1
        elif abs_diff <= 3: matched_counts["close_3s"] += 1
        else: matched_counts["far"] += 1

    # 3. Generate Report
    avg_offset = np.mean(deltas)
    median_offset = np.median(deltas)

    print("\n" + "="*40)
    print("TIME SHIFT DIAGNOSTICS")
    print("="*40)
    print(f"Mean offset (Cam2 - Cam1): {avg_offset:.2f} sec")
    print(f"Median offset: {median_offset:.2f} sec")
    print("-" * 40)
    print("Match distribution:")
    print(f"Exact (0s):       {matched_counts['exact']}")
    print(f"Very close (1s):  {matched_counts['close_1s']}")
    print(f"Close (<=3s):     {matched_counts['close_3s']}")
    print(f"Far (>3s):        {matched_counts['far']}")
    print("-" * 40)

    if abs(median_offset) > 3600:
        print("Looks like a timezone error (difference > 1h).")
    elif abs(median_offset) > 10:
        print("Camera clocks are desynchronized by a constant value.")
    elif matched_counts['close_1s'] > 100:
        print("Cameras are synchronized, but have a small delay (<1s).")
        print("   You must accept matching with +/- 1s tolerance.")

if __name__ == "__main__":
    main()

In [ ]:
DATASET_DIR = Path(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\Processed_Dataset")
OUTPUT_DIR = Path(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\Szymon_dataset_and_plot")

def main():
    print(f"Analyzing dataset at {DATASET_DIR}")

    if not DATASET_DIR.exists():
        print("Error: Dataset directory not found.")
        return

    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    hours = []
    total_images = 0

    for img_path in DATASET_DIR.rglob("*.jpg"):
        total_images += 1
        stem = img_path.stem.split('_')[0]

        if stem.isdigit():
            timestamp = int(stem)
            dt = datetime.fromtimestamp(timestamp)
            hours.append(dt.hour)

    print(f"Found {total_images} images.")

    if total_images == 0:
        return

    print("Generating Histogram...")

    plt.figure(figsize=(10, 6))
    plt.hist(hours, bins=range(25), color='#2c3e50', edgecolor='white', alpha=0.8, align='left')

    plt.title(f"Dataset Distribution by Hour of Day (n={total_images})", fontsize=14)
    plt.xlabel("Hour of Day (0-23)", fontsize=12)
    plt.ylabel("Number of Images", fontsize=12)
    plt.xticks(range(0, 24))
    plt.grid(axis='y', alpha=0.3)

    plot_path = OUTPUT_DIR / "dataset_hour_histogram.png"
    plt.savefig(plot_path, dpi=300)
    print(f"Histogram saved to: {plot_path}")

    print("\nCreating ZIP archive...")
    zip_name = OUTPUT_DIR / "final_dataset"
    shutil.make_archive(zip_name, 'zip', DATASET_DIR)

    print(f"SUCCESS: Dataset compressed to {zip_name}.zip")

if __name__ == "__main__":
    main()

In [ ]:
DATASET_DIR = Path(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\Processed_Dataset")
OUTPUT_DIR = Path(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\Szymon_dataset_and_plot")

def main():
    print("Generating Plot (8-17 range)...")

    if not OUTPUT_DIR.exists():
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    if not DATASET_DIR.exists():
        print("Error: Dataset folder not found.")
        return

    hours = []
    total_images = 0

    for img_path in DATASET_DIR.rglob("*.jpg"):
        total_images += 1
        stem = img_path.stem.split('_')[0]
        if stem.isdigit():
            timestamp = int(stem)
            dt = datetime.fromtimestamp(timestamp)
            hours.append(dt.hour)

    if total_images == 0:
        print("No images found.")
        return

    print("Drawing histogram...")

    plt.figure(figsize=(10, 6))

    bins = np.arange(0, 25) - 0.5

    plt.hist(hours, bins=bins, color='#4c72b0', edgecolor='black', alpha=0.9, rwidth=0.8, zorder=3)

    target_hours = range(8, 18)
    hour_labels = [f"{h:02d}:00" for h in target_hours]

    plt.xticks(ticks=target_hours, labels=hour_labels, rotation=45, ha='right', fontsize=11)
    plt.xlim([7.5, 17.5])

    plt.xlabel("Time of Day (Local Time)", fontsize=12)
    plt.ylabel("Image Count", fontsize=12)

    plt.grid(axis='y', linestyle='--', alpha=0.5, zorder=0)
    plt.tight_layout()

    plot_path = OUTPUT_DIR / "dataset_hour_histogram_8_17.png"
    plt.savefig(plot_path, dpi=300)

    print(f"Done! Plot saved to:\n{plot_path}")

if __name__ == "__main__":
    main()

In [ ]:
# Input: Where your current raw folders are (camera_1, camera_2)
SOURCE_ROOT = Path(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo")

# Output: Where the new, organized dataset will be created
OUTPUT_ROOT = Path(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Processed_Dataset")

SUBSETS = ["train", "test", "valid"]

# Mapping raw folder names to Article/Display names
NAME_MAPPING = {
    "camera_1": "East pylon",
    "camera_2": "Sprogoe"
}

# Metadata descriptions
FOOTPRINTS = {
    "East pylon": "Loc: East Bridge Pylon, View: High-angle Roadway & Cables, Dir: West (towards Sprogoe)",
    "Sprogoe":    "Loc: Sprogoe/Coast, View: Bridge Profile (Side View), Dir: East/South-East"
}

# Regex to find date/time in original filenames (e.g., frame_20231005_120000)
FILENAME_PATTERN = re.compile(r"frame_(\d{8})_(\d{6})_")

def get_unique_path(directory, filename, ext):
    """Ensures we don't overwrite files if timestamps are identical."""
    base = filename
    counter = 0
    while True:
        suffix = f"_{counter}" if counter > 0 else ""
        full_path = directory / f"{base}{suffix}{ext}"
        if not full_path.exists():
            return full_path

def main():
    metadata_records = []
    print(f"Generating dataset at: {OUTPUT_ROOT}")

    for source_folder_name, new_camera_name in NAME_MAPPING.items():
        print(f"Processing: '{source_folder_name}' -> '{new_camera_name}'")

        footprint = FOOTPRINTS.get(new_camera_name, "Unknown View")

        for subset in SUBSETS:
            img_src_dir = SOURCE_ROOT / source_folder_name / subset / "images"
            lbl_src_dir = SOURCE_ROOT / source_folder_name / subset / "labels"

            if not img_src_dir.exists():
                print(f"   Skipped (missing): {img_src_dir}")
                continue

            for src_img in img_src_dir.rglob("*.jpg"):
                # Extract date from filename
                match = FILENAME_PATTERN.search(src_img.name)
                if not match:
                    continue

                d_str, t_str = match.groups()
                dt = datetime.strptime(f"{d_str}{t_str}", "%Y%m%d%H%M%S")
                timestamp = int(dt.timestamp())
                date_str = dt.strftime("%Y-%m-%d")

                # Define new path structure
                target_dir = OUTPUT_ROOT / new_camera_name / date_str
                (target_dir / "images").mkdir(parents=True, exist_ok=True)
                (target_dir / "labels").mkdir(parents=True, exist_ok=True)

                # Copy and Rename Image to Timestamp
                dest_img = get_unique_path(target_dir / "images", str(timestamp), ".jpg")
                shutil.copy2(src_img, dest_img)
                final_name = dest_img.name

                # Process Label if it exists
                src_lbl = lbl_src_dir / (src_img.stem + ".txt")
                ship_count = 0
                if src_lbl.exists():
                    dest_lbl = target_dir / "labels" / final_name.replace(".jpg", ".txt")
                    shutil.copy2(src_lbl, dest_lbl)
                    
                    with open(src_lbl, 'r') as f:
                        ship_count = len([line for line in f if line.strip()])

                # Collect Metadata
                metadata_records.append({
                    "filename": final_name,
                    "camera": new_camera_name,
                    "split": subset,
                    "date": date_str,
                    "unix_timestamp": timestamp,
                    "ship_count": ship_count,
                    "view_footprint": footprint
                })

    # Save Master CSV
    if metadata_records:
        df = pd.DataFrame(metadata_records)
        df = df[["filename", "split", "camera", "date", "ship_count", "unix_timestamp", "view_footprint"]]

        meta_path = OUTPUT_ROOT / "dataset_metadata.csv"
        df.to_csv(meta_path, index=False)
        print(f"\nSUCCESS: Metadata saved to:\n{meta_path}")
        print("\nPreview:")
        print(df[['filename', 'camera', 'split', 'ship_count']].head().to_string(index=False))
    else:
        print("ERROR: No files found. Check NAME_MAPPING.")

if __name__ == "__main__":
    main()

In [ ]:
# ================= CONFIGURATION =================
META_FILE = Path(r"C:\Users\szymo\Desktop\DTU\3rd_semester\final_dataset\dataset_metadata.csv")
OUTPUT_DIR = Path(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\Szymon_dataset_and_plot")

# Days to filter (Format: 'MM-DD')
TARGET_DAYS = ['01-08', '12-14']

def main():
    if not META_FILE.exists():
        print("Error: Metadata file not found.")
        return
    
    if not OUTPUT_DIR.exists():
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    # 1. Load Data
    df = pd.read_csv(META_FILE)
    df['dt'] = pd.to_datetime(df['unix_timestamp'], unit='s')

    # 2. Filter by Date (e.g., Jan 8 and Dec 14)
    df['month_day'] = df['dt'].dt.strftime('%m-%d')
    df_filtered = df[df['month_day'].isin(TARGET_DAYS)].copy()

    if df_filtered.empty:
        print(f"WARNING: No images found for days: {TARGET_DAYS}")
        print("Available dates in file:", df['date'].unique())
        return
    else:
        print(f"Found {len(df_filtered)} images for days: {TARGET_DAYS}")

    # 3. Prepare Plot Data
    # X-Axis: Normalize everything to a single arbitrary date to compare times
    df_filtered['time_of_day'] = df_filtered['dt'].apply(lambda x: x.replace(year=1900, month=1, day=1))

    # Y-Axis: Sort chronologically so dates appear in order
    df_filtered = df_filtered.sort_values(by=['date', 'camera'], ascending=[True, False])
    df_filtered['y_label'] = df_filtered['date'] + "  /  " + df_filtered['camera']

    # 4. Drawing
    print("Generating plot...")
    plt.figure(figsize=(12, 5))

    colors = {
        'East pylon': '#003366',  # Dark Blue
        'Sprogoe':    '#66b3ff'   # Light Blue
    }

    y_labels = df_filtered['y_label'].unique()

    for label in y_labels:
        subset = df_filtered[df_filtered['y_label'] == label]
        if subset.empty: continue

        cam_name = subset['camera'].iloc[0]
        col = colors.get(cam_name, 'gray')

        # Strip Plot using scatter
        plt.scatter(
            subset['time_of_day'],
            [label] * len(subset),
            marker='|',
            s=300,
            color=col,
            alpha=0.8,
            linewidths=1.5
        )

    # 5. Formatting
    ax = plt.gca()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))

    plt.xlabel("Time of Day (UTC / Local)", fontsize=11)
    
    plt.grid(axis='x', linestyle='--', alpha=0.5)
    plt.grid(axis='y', linestyle='-', alpha=0.2)

    plt.tight_layout()

    save_path = OUTPUT_DIR / "dataset_strip_plot.png"
    plt.savefig(save_path, dpi=300)
    print(f"Plot saved to: {save_path}")

if __name__ == "__main__":
    main()

## Crosseval

In [ ]:
# --- CONFIGURATION ---
BASE_DIR = r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\cross_mix"
CSV_PATH = os.path.join(BASE_DIR, "dataset_metadata.csv")

# Key: Name in CSV 'camera' column
# Value: Actual folder name on disk
CAMERA_FOLDER_MAP = {
    'East pylon': 'East Pylon',
    'Sprogoe': 'Sprogoe'
}

def generate_configs_for_nested_structure():
    if not os.path.exists(CSV_PATH):
        raise FileNotFoundError(f"dataset_metadata.csv not found at {CSV_PATH}")

    df = pd.read_csv(CSV_PATH)

    for csv_cam, folder_cam in CAMERA_FOLDER_MAP.items():
        print(f"Processing camera: {csv_cam} (Folder: {folder_cam})...")

        subset = df[df['camera'] == csv_cam]

        if subset.empty:
            print(f"WARNING: No images found in CSV for camera '{csv_cam}'!")
            continue

        file_paths = {'train': [], 'valid': [], 'test': []}

        for index, row in subset.iterrows():
            split = row['split']
            date_folder = str(row['date'])
            filename = row['filename']

            # Path structure: cross_mix / Camera / Date / images / Filename
            full_path = os.path.join(BASE_DIR, folder_cam, date_folder, 'images', filename)
            
            # Using absolute path for safety
            file_paths[split].append(os.path.abspath(full_path))

        txt_files_map = {}
        for split_name, paths in file_paths.items():
            if not paths:
                continue

            txt_filename = f"{folder_cam.replace(' ', '_').lower()}_{split_name}.txt"
            with open(txt_filename, 'w') as f:
                f.write('\n'.join(paths))

            txt_files_map[split_name] = os.path.abspath(txt_filename)
            print(f"   -> {split_name}: found {len(paths)} images -> saved to {txt_filename}")

        # YOLO expects 'val', CSV often has 'valid'
        val_path = txt_files_map.get('valid') or txt_files_map.get('val')

        yaml_content = f"""
path: {BASE_DIR}
train: {txt_files_map.get('train')}
val: {val_path}
test: {txt_files_map.get('test')}

names:
  0: ship
"""
        yaml_name = f"{folder_cam.replace(' ', '_').lower()}.yaml"
        with open(yaml_name, 'w') as f:
            f.write(yaml_content)

        print(f"   -> DONE: Config file created: {yaml_name}\n")

if __name__ == "__main__":
    generate_configs_for_nested_structure()

## 

In [ ]:
model = YOLO(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\runs\detect\train4\weights\best.pt")

print("--- WYNIKI DLA EAST PYLON (Test Set) ---")
res_ep = model.val(data="east_pylon.yaml", split='test')

print("\n--- WYNIKI DLA SPROGOE (Test Set) ---")
res_sp = model.val(data="sprogoe.yaml", split='test')

### Mixed crosscamera experiment

In [ ]:
# --- CONFIGURATION ---
BASE_MODEL = r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\runs\detect\train26\weights\best.pt"
SPROGOE_DIR = Path(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\cross_mix\East pylon")

WORK_DIR = Path("experiment_sprogoe_to_eastpylon")
IMG_SIZE = 1280
BATCH_SIZE = 4
EPOCHS = 15
FRACTIONS = [0.0, 0.10, 0.20, 0.30, 0.40, 0.50]

AUG_PARAMS = {
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.6,
    'degrees': 5.0,
    'translate': 0.1,
    'scale': 0.5,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.1,
}

def setup_directories():
    if WORK_DIR.exists():
        try:
            shutil.rmtree(WORK_DIR)
        except Exception as e:
            print(f"Error: Could not remove {WORK_DIR}: {e}")
    WORK_DIR.mkdir(parents=True, exist_ok=True)

def get_data_recursive(source_root):
    pairs = []
    print(f"Scanning: {source_root}")

    all_images = (list(source_root.rglob("*.jpg")) +
                  list(source_root.rglob("*.png")) +
                  list(source_root.rglob("*.jpeg")))

    for img_path in all_images:
        if img_path.parent.name == "images":
            lbl_path = img_path.parent.parent / "labels" / (img_path.stem + ".txt")
        else:
            lbl_path = img_path.with_suffix(".txt")

        if lbl_path.exists():
            pairs.append((img_path, lbl_path))

    print(f"Found {len(pairs)} pairs")
    return pairs

def create_dataset_yaml(name, train_pairs, test_pairs):
    base = WORK_DIR / name
    dirs = {
        'train_img': base / "images" / "train",
        'train_lbl': base / "labels" / "train",
        'test_img': base / "images" / "test",
        'test_lbl': base / "labels" / "test"
    }

    for d in dirs.values():
        d.mkdir(parents=True, exist_ok=True)

    for img, lbl in train_pairs:
        shutil.copy2(img, dirs['train_img'] / img.name)
        shutil.copy2(lbl, dirs['train_lbl'] / lbl.name)

    for img, lbl in test_pairs:
        shutil.copy2(img, dirs['test_img'] / img.name)
        shutil.copy2(lbl, dirs['test_lbl'] / lbl.name)

    yaml_content = {
        'path': str(base.resolve()),
        'train': 'images/train',
        'val': 'images/test',
        'test': 'images/test',
        'names': {0: 'ship'}
    }

    yaml_path = base / "data.yaml"
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)

    return str(yaml_path)

def main():
    print("Starting Experiment: Pretrain (SMD+SeaShips+EastPylon) -> Fine-tune Sprogoe")
    setup_directories()

    sprogoe_data = get_data_recursive(SPROGOE_DIR)
    if not sprogoe_data:
        print("Error: No data found in Sprogoe directory")
        return

    random.seed(42)
    random.shuffle(sprogoe_data)

    results = []

    for frac in FRACTIONS:
        pct = int(frac * 100)
        n_finetune = int(len(sprogoe_data) * frac)
        finetune_set = sprogoe_data[:n_finetune]
        eval_set = sprogoe_data[n_finetune:]

        print(f"Processing Fine-tune: {pct}% | Eval: {100-pct}%")
        exp_name = f"EastPylon_to_Sprogoe_ft{pct}"

        try:
            yaml_path = create_dataset_yaml(exp_name, finetune_set, eval_set)
            model = YOLO(BASE_MODEL)

            if pct == 0:
                print("Running Zero-shot evaluation")
                res = model.val(data=yaml_path, split='test', imgsz=IMG_SIZE, batch=BATCH_SIZE, verbose=False)
            else:
                print(f"Running Fine-tuning for {pct}%")
                model.train(
                    data=yaml_path,
                    epochs=EPOCHS,
                    imgsz=IMG_SIZE,
                    batch=BATCH_SIZE,
                    workers=4,
                    project=str(WORK_DIR / "runs"),
                    name=exp_name,
                    verbose=False,
                    patience=50,
                    save=True,
                    exist_ok=True,
                    **AUG_PARAMS
                )
                res = model.val(split='test', verbose=False)

            metrics = {
                'pretrain': 'EastPylon',
                'target': 'Sprogoe',
                'finetune_pct': pct,
                'mAP50': float(res.box.map50),
                'mAP50-95': float(res.box.map)
            }
            results.append(metrics)
            print(f"Results for {pct}%: mAP50={metrics['mAP50']:.4f}")

        except Exception as e:
            print(f"Error during execution: {e}")

    # Export results
    df = pd.DataFrame(results)
    df.to_csv(WORK_DIR / "results.csv", index=False)

    # Generate plots
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].plot(df['finetune_pct'], df['mAP50'], marker='o', color='red')
    axes[0].set_title('mAP@0.5')
    axes[0].grid(True)

    axes[1].plot(df['finetune_pct'], df['mAP50-95'], marker='o', color='red')
    axes[1].set_title('mAP@0.5:0.95')
    axes[1].grid(True)

    plt.savefig(WORK_DIR / "results_plot.png")
    print("Experiment finished. Results saved.")

if __name__ == "__main__":
    main()

In [ ]:
# ================= CONFIGURATION =================
CSV1 = "experiment_eastpylon_to_sprogoe/results.csv"
CSV2 = "experiment_sprogoe_to_eastpylon/results.csv"

LABEL1 = "EastPylon -> Sprogoe"
LABEL2 = "SMD -> Sprogoe"

# ================= DATA LOADING =================
df1 = pd.read_csv(CSV1)
df2 = pd.read_csv(CSV2)

# ================= PLOTTING =================
plt.style.use('seaborn-v0_8-darkgrid')
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# --- Plot 1: mAP@0.5 ---
axes[0].plot(df1['finetune_pct'], df1['mAP50'],
             marker='o', linewidth=3, markersize=10,
             color='#e74c3c', label=LABEL1, alpha=0.9)
axes[0].plot(df2['finetune_pct'], df2['mAP50'],
             marker='s', linewidth=3, markersize=10,
             color='#3498db', label=LABEL2, alpha=0.9)

axes[0].set_xlabel('Fine-tuning data (%)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('mAP@0.5', fontsize=14, fontweight='bold')
axes[0].set_title('Transfer Learning Comparison: mAP@0.5', fontsize=16, fontweight='bold')
axes[0].legend(fontsize=12, loc='lower right', frameon=True, shadow=True)
axes[0].grid(True, alpha=0.3, linestyle='--')
axes[0].set_ylim([0, 1])

# --- Plot 2: mAP@0.5:0.95 ---
axes[1].plot(df1['finetune_pct'], df1['mAP50-95'],
             marker='o', linewidth=3, markersize=10,
             color='#e74c3c', label=LABEL1, alpha=0.9)
axes[1].plot(df2['finetune_pct'], df2['mAP50-95'],
             marker='s', linewidth=3, markersize=10,
             color='#3498db', label=LABEL2, alpha=0.9)

axes[1].set_xlabel('Fine-tuning data (%)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('mAP@0.5:0.95', fontsize=14, fontweight='bold')
axes[1].set_title('Transfer Learning Comparison: mAP@0.5:0.95', fontsize=16, fontweight='bold')
axes[1].legend(fontsize=12, loc='lower right', frameon=True, shadow=True)
axes[1].grid(True, alpha=0.3, linestyle='--')
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.savefig('comparison_plot.png', dpi=300, bbox_inches='tight')
print("Comparison plot saved: comparison_plot.png")
plt.show()

## Trying the code on the photos from the whole day

In [ ]:
# 1. Load your trained YOLO model
model = YOLO(r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\experiment_crossmix_full_metrics_eastpylon\runs\ft_50\weights\best.pt")

# 2. Define paths
#input_folder = r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\sprogo_first_colab_frames\14_01_2026\frames_sb2"
input_folder1 = r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\sprogo_first_colab_frames\14_01_2026\frames_sb1"
#input_folder2 = r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\sprogo_first_colab_frames\14_01_2026\frames_sb3"
#output_folder = r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\sprogo_first_colab_frames\14_01_2026\frames_sb2_detect"
output_folder2 = r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\sprogo_first_colab_frames\14_01_2026\frames_sb1_detect"
#output_folder3 = r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\sprogo_first_colab_frames\14_01_2026\frames_sb3_detect"

# 3. Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# 4. Define water region; allow configurable fraction (e.g., 0.6 => bottom 60%)
def create_water_mask(img_shape, water_percent=0.6):
    """Create a binary mask for the water region.

    water_percent: fraction (0.0-1.0) of image height to consider as "water" from the bottom.
    If water_percent==1.0 the entire image is considered.
    """
    height, width = img_shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)

    # Clamp percent and compute start row
    water_percent = max(0.0, min(1.0, float(water_percent)))
    water_start_y = int(height * (1.0 - water_percent))
    mask[water_start_y:, :] = 255

    return mask

def filter_detections_in_roi(boxes, mask):
    """Filter detections to only include those whose center lies in the mask."""
    filtered_indices = []

    for idx, box in enumerate(boxes):
        # Get bounding box coordinates
        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())

        # Calculate center point of bounding box
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        # Bounds check before indexing mask
        if 0 <= center_y < mask.shape[0] and 0 <= center_x < mask.shape[1]:
            if mask[center_y, center_x] > 0:
                filtered_indices.append(idx)

    return filtered_indices

# 5. Get all image files from both input folders and process each
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']

# Ensure both output folders exist
#os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder2, exist_ok=True)
#os.makedirs(output_folder3, exist_ok=True)

# Process both input folders and save to corresponding output folders
folders = [
    #(input_folder, output_folder),
    (input_folder1, output_folder2),
    #(input_folder2, output_folder3)
]

for in_folder, out_folder in folders:
    try:
        image_files = [f for f in os.listdir(in_folder)
                       if os.path.splitext(f)[1].lower() in image_extensions]
    except FileNotFoundError:
        print(f"Input folder not found: {in_folder}")
        continue

    # Set per-folder confidence threshold and water region:
    # - input_folder (frames_sb2): analyze whole image, conf=0.55
    # - input_folder1 (frames_sb1): analyze bottom 60%, conf=0.60
    if in_folder == input_folder:
        conf_thr = 0.65
        water_percent = 0.6
    else:
        conf_thr = 0.60
        water_percent = 0.60

    print(f"Found {len(image_files)} images in {in_folder} to process -> {out_folder} (conf={conf_thr}, water_percent={water_percent})")

    for img_file in image_files:
        img_path = os.path.join(in_folder, img_file)

        # Load image to get dimensions and create mask
        img = cv2.imread(img_path)
        if img is None:
            print(f"Could not read: {img_file} in {in_folder}")
            continue

        # Create water mask according to folder-specific percentage
        water_mask = create_water_mask(img.shape, water_percent=water_percent)

        # Run prediction on full image with per-folder confidence
        results = model.predict(
            source=img_path,
            save=False,
            show=False,
            conf=conf_thr,
            iou=0.0
        )

        # Filter detections to only water region
        if len(results[0].boxes) > 0:
            valid_indices = filter_detections_in_roi(results[0].boxes, water_mask)

            if valid_indices:
                # Keep only detections in water region
                results[0].boxes = results[0].boxes[valid_indices]

                # Save the image with filtered detections
                output_path = os.path.join(out_folder, img_file)
                results[0].save(filename=output_path)
                print(f"Saved: {img_file} ({len(valid_indices)} detections in water) -> {out_folder}")
            else:
                print(f"Skipped: {img_file} (no detections in water region) in {in_folder}")
        else:
            pass  # No detections at all

print("\nProcessing complete! Detected images saved to:")
print(f" - {output_folder}")
print(f" - {output_folder2}")
#print(f" - {output_folder3}")
print("Analyzing bottom 60% of each image (water region)")


### Ground truth and detection (for the report)

In [ ]:
# --- PATH CONFIGURATION ---
# Provide paths to your files:
IMAGE_PATH = r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\camera_2\train\images\frame_20251214_150720_jpg.rf.cacb49ad7037e8f913d1b7fb17b367fa.jpg"  # Your image
LABEL_PATH = r"C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\camera_2\train\labels\frame_20251214_150720_jpg.rf.cacb49ad7037e8f913d1b7fb17b367fa.txt"   # Ground Truth txt file (YOLO format)
MODEL_PATH = r'C:\Users\szymo\Desktop\DTU\3rd_semester\Individual_project_demo\runs\detect\train4\weights\best.pt'                               # Path to your trained .pt model
OUTPUT_PATH = '2nd_camera_detect_simple.jpg'         # Output file destination

def draw_yolo_box(img, line, color, label_text):
    """
    Draws a bounding box from normalized YOLO format onto the image.
    YOLO format in txt: class x_center y_center width height
    """
    h, w, _ = img.shape
    parts = line.strip().split()

    # Data parsing (assuming class_id is the first element, followed by coordinates)
    # class_id = int(parts[0])
    x_center, y_center, box_w, box_h = map(float, parts[1:5])

    # Convert from relative (0-1) to pixel coordinates
    x_c, y_c = x_center * w, y_center * h
    b_w, b_h = box_w * w, box_h * h

    # Calculate top-left corner (x_min, y_min) and bottom-right corner
    x_min = int(x_c - (b_w / 2))
    y_min = int(y_c - (b_h / 2))
    x_max = int(x_c + (b_w / 2))
    y_max = int(y_c + (b_h / 2))

    # Draw the rectangle
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, 3)

    # Add label text
    cv2.putText(img, label_text, (x_min, y_min - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# --- 1. IMAGE LOADING ---
image = cv2.imread(IMAGE_PATH)
if image is None:
    print(f"Error: Image not found at {IMAGE_PATH}")
    exit()

image_vis = image.copy() # Copy for visualization/drawing

# --- 2. DRAW GROUND TRUTH (GREEN) ---
# Reading the txt file (typically exported from Roboflow)
if os.path.exists(LABEL_PATH):
    with open(LABEL_PATH, 'r') as f:
        lines = f.readlines()
        for line in lines:
            # Drawing the GT box (BGR color (0, 255, 0) is Green)
            draw_yolo_box(image_vis, line, (0, 255, 0), "Ground Truth")
else:
    print("Warning: Ground Truth file (.txt) not found")

# --- 3. DRAW MODEL PREDICTIONS (RED) ---
print("Running model inference...")
model = YOLO(MODEL_PATH)
results = model.predict(IMAGE_PATH, conf=0.25) # conf - confidence threshold

for result in results:
    boxes = result.boxes
    for box in boxes:
        # Get xyxy coordinates (x_min, y_min, x_max, y_max)
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        conf = box.conf[0].item()

        # Draw Prediction box (BGR color (0, 0, 255) is Red)
        cv2.rectangle(image_vis, (x1, y1), (x2, y2), (0, 0, 255), 3)

        label = f"Pred: {conf:.2f}"
        # Position label below the box to avoid overlapping with GT labels
        cv2.putText(image_vis, label, (x1, y2 + 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

# --- 4. DISPLAY AND SAVE ---
# Convert BGR (OpenCV default) to RGB (Matplotlib default) for correct color rendering
image_rgb = cv2.cvtColor(image_vis, cv2.COLOR_BGR2RGB)



plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.axis('off')
plt.title("Green: Ground Truth | Red: Model Prediction")
plt.show()

# Save output to disk
cv2.imwrite(OUTPUT_PATH, image_vis)
print(f"Resulting image saved as: {OUTPUT_PATH}")